In [ ]:
!pip install numpy kaggle pandas pyspark findspark matplotlib scikit-learn plotly

import os
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import numpy as np
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, col, udf, rand
import matplotlib.pyplot as plt
import math
import pyspark.sql as ps
from zlib import crc32
import time as tm
from datetime import datetime as dt
from dateutil import parser
import itertools
from collections import defaultdict
from dataclasses import dataclass
from pyspark.sql import functions as F
from pyspark.rdd import RDD
from pyspark.broadcast import Broadcast
import findspark

In [ ]:
path = './data'
worker_nodes = "*"
problem_to_solve = 'DIVERTED'

dataset_limit = 100000
use_all_dataset_frames = True
fold_number = 10
load_cached = True

def print_and_save_time(s: str):
  f = open("./times.txt", "a")
  f.write(str(dt.now()) + " - " + s + "\n")
  f.close()
  print(s)

## Data Download

In [ ]:
os.environ['KAGGLE_USERNAME'] = "davidetricella"
os.environ['KAGGLE_KEY'] = "e1ab3aae4a07f36b37a3a8bace74d9df"


dataset = 'yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018'
path = './data'

In [ ]:
def download_dataset():
    if not os.path.isdir(path):
        os.mkdir(path)
    if not os.listdir(path):
        try:
            api = KaggleApi()
            api.authenticate()
            api.dataset_download_files(dataset, path, unzip=True, quiet=False)
        except:
            print("Error downloading the dataset")

download_dataset()

## Data Loading

In [ ]:
dataframe_schema = StructType([
    StructField('FL_DATE', StringType(), True),
    StructField('OP_CARRIER', StringType(), True),
    StructField('ORIGIN', StringType(), True),
    StructField('DEST', StringType(), True),
    StructField('CRS_DEP_TIME', StringType(), True),
    StructField('CRS_ARR_TIME', StringType(), True),
    StructField('CANCELLED', StringType(), True),
    StructField('DIVERTED', StringType(), True),
    StructField('CRS_ELAPSED_TIME', StringType(), True),
    StructField('DISTANCE', StringType(), True)
])

columns_to_get = [
    'FL_DATE',
    'OP_CARRIER',
    'ORIGIN',
    'DEST',
    'CRS_DEP_TIME',
    'CRS_ARR_TIME',
    'CANCELLED',
    'DIVERTED',
    'CRS_ELAPSED_TIME',
    'DISTANCE'
]

#Rounded max distance between airports found in the dataset
max_distance = 4970

findspark.init()
findspark.find()

spark = SparkSession.builder \
.appName("Airline Departure") \
.master('local[' + worker_nodes + ']') \
.getOrCreate()

context = spark.sparkContext

#The rows are loaded fom all files or only the first one with a limit value
def get_dataset(limit: float = -1, allFrames: bool = True):
    files = os.listdir(path)
    big_frame = spark.createDataFrame(
        spark.sparkContext.emptyRDD(), schema=dataframe_schema)
    if not allFrames:
        files = [files[0]]

    for f in files:
        if f.endswith('.csv'):
            frame = spark.read.option("header", True).csv(path + '/' + f)
            frame = frame.select(columns_to_get)
            frame = frame.orderBy(rand())

            if limit != -1:
                frame = frame.limit(limit)

            big_frame = frame.union(big_frame)

    big_frame = big_frame.withColumn(
        "id", monotonically_increasing_id()).orderBy(rand())
    big_frame.count()

    return big_frame

## Preprocessing

#### Chart Plotting Definitions

These are the defintions of the charts plotted during data analysis, the main chart used is the bar charts, to have a grasp of the 
various distributions of problematic flights regarding the most important columns chosen to train the model.
During charts plotting, the data has been collected, taking advantage of the really low numbers of problematic flights in the dataset.

In [ ]:
def month_key_to_label(key):
  match key:
    case 1:
        return 'JAN'
    case 2:
        return 'FEB'
    case 3:
        return 'MAR'
    case 4:
        return 'APR'
    case 5:
        return 'MAY'
    case 6:
        return 'JUN'
    case 7:
        return 'JUL'
    case 8:
        return 'AUG'
    case 9:
        return 'SEP'
    case 10:
        return 'OCT'
    case 11:
        return 'NOV'
    case 12:
        return 'DEC'

def plot_balancing_chart(data: ps.DataFrame, label: str):
  total_positives = data.filter(col(label) == 1).count()
  total_negatives = data.filter(col(label) == 0).count()
  fig, ax = plt.subplots()

  labels = ['REGULAR', label]
  counts = [total_negatives, total_positives]
  bar_colors = ['tab:blue', 'tab:red']

  cont = ax.bar(labels, counts, color=bar_colors)
  ax.bar_label(container=cont)

  ax.set_ylabel('Counts')
  ax.set_title("Regular flights and " + problem_to_solve.lower() + " flights counts")

  plt.show()

def plot_problematic_flights_per_month_chart(data: ps.DataFrame, label: str):
  problematic_flights = data.filter(col(label) == 1).collect()
  fig, ax = plt.subplots()

  flights_dict = defaultdict(int)
  labels = []
  counts = []

  for row in problematic_flights:
    month = parser.parse(row["FL_DATE"]).month
    flights_dict[month] += 1

  for key in flights_dict:
    condition_string = ("-0" + str(key) + "-") if key < 10 else ("-" + str(key) + "-")
    single_month_total_flights = data.filter(col("FL_DATE").contains(condition_string)).count()
    flights_dict[key] = flights_dict[key] / single_month_total_flights * 100

  for key in sorted(flights_dict):
    labels.append(month_key_to_label(key))
    counts.append(flights_dict[key])

  ax.bar(labels, counts)

  ax.set_ylabel('Percentages')
  ax.set_xlabel('Months')
  ax.set_title(problem_to_solve.lower().capitalize() + ' flights percentage per month')

  plt.show()

def plot_problematic_flights_per_weekday_chart(data: ps.DataFrame, label: str):
  problematic_flights = data.filter(col(label) == 1).collect()
  fig, ax = plt.subplots()

  flights_dict = defaultdict(int)
  days_dict = defaultdict(int)
  labels = []
  counts = []

  for row in problematic_flights:
    weekday = parser.parse(row["FL_DATE"]).isoweekday()
    flights_dict[weekday] += 1    

  dates_frame = data.select("FL_DATE").collect()

  for row in dates_frame:
      parsed_value = parser.parse(row["FL_DATE"]).isoweekday()
      days_dict[parsed_value] += 1

  for key in flights_dict:
    flights_dict[key] = flights_dict[key] / days_dict[key] * 100

  for key in sorted(flights_dict):
    labels.append(key)
    counts.append(flights_dict[key])

  ax.bar(labels, counts)

  ax.set_ylabel('Percentages')
  ax.set_xlabel('Weekdays')
  ax.set_title(problem_to_solve.lower().capitalize() + ' flights percentage per weekday')

  plt.show()

def plot_problematic_flights_per_carrier_chart(data: ps.DataFrame, label: str):
  selected_data = data.select((col(problem_to_solve).cast('double')).alias(problem_to_solve),
                              (data.OP_CARRIER).alias("OP_CARRIER"))
  flight_counts = selected_data.groupBy("OP_CARRIER").count().collect()
  problematic_flights = selected_data.groupBy("OP_CARRIER").sum(problem_to_solve).collect()

  fig, ax = plt.subplots()

  flights_dict = dict()
  labels = []
  counts = []

  for i in range(len(problematic_flights)):
    flights_dict[problematic_flights[i][0]] = problematic_flights[i][1] / flight_counts[i][1] * 100

  mean = sum(flights_dict.values()) / len(flights_dict)

  for key in flights_dict:
    if flights_dict[key] > mean:
      labels.append(key)
      counts.append(flights_dict[key])
  labels.append("Mean")
  counts.append(mean)

  ax.bar(labels, counts)

  ax.set_ylabel('Percentages')
  ax.set_xlabel('Carrier codes')
  ax.set_title(problem_to_solve.lower().capitalize() +
               ' flights percentage per carrier')

  plt.show()

def plot_problematic_flights_per_origin_box(data: ps.DataFrame, label: str):
  selected_data = data.select((col(problem_to_solve).cast('double')).alias(problem_to_solve),
                              (data.ORIGIN).alias("ORIGIN"))
  flight_counts = selected_data.groupBy("ORIGIN").count().collect()
  problematic_flights = selected_data.groupBy(
      "ORIGIN").sum(problem_to_solve).collect()

  fig, ax = plt.subplots()

  flights_dict = dict()
  labels = []
  counts = []

  for i in range(len(problematic_flights)):
    flights_dict[problematic_flights[i][0]
                 ] = problematic_flights[i][1] / flight_counts[i][1] * 100

  for key in flights_dict:
    labels.append(key)
    counts.append(flights_dict[key])

  ax.boxplot(counts)

  ax.set_ylabel('Percentages')
  ax.set_xlabel('Airport codes')
  ax.set_title(problem_to_solve.lower().capitalize() +
               ' flights percentage per origin')

  plt.show()

def plot_problematic_flights_per_origin_chart(data: ps.DataFrame, label: str):
  selected_data = data.select((col(problem_to_solve).cast('double')).alias(problem_to_solve),
                              (data.ORIGIN).alias("ORIGIN"))
  flight_counts = selected_data.groupBy("ORIGIN").count().collect()
  problematic_flights = selected_data.groupBy(
      "ORIGIN").sum(problem_to_solve).collect()

  fig, ax = plt.subplots()
  fig.set_size_inches(18.5, 10.5)

  flights_dict = dict()
  labels = []
  counts = []

  for i in range(len(problematic_flights)):
    flights_dict[problematic_flights[i][0]
                 ] = problematic_flights[i][1] / flight_counts[i][1] * 100

  for key in flights_dict:
    condition = 5 if problem_to_solve == 'CANCELLED' else 1
    if flights_dict[key] > condition:
      labels.append(key)
      counts.append(flights_dict[key])
  labels.append("Majority")
  counts.append(1)

  ax.bar(labels, counts)

  ax.set_ylabel('Percentages')
  ax.set_title(problem_to_solve.lower().capitalize() +
               ' flights percentage per origin')

  plt.show()


#### Dataset Reading

In [ ]:
start_time = tm.time()
data = get_dataset(dataset_limit, use_all_dataset_frames).cache()

finish_time = tm.time() - start_time
print_and_save_time("Dataset reading concluded: " +
                    str(finish_time) + " seconds")

#### Null Rows Dropping

Various tests showed that the rows with null values are extremely rare, so removing them form the dataset will not bring any significative loss of informations.


In [ ]:
common_start_time = tm.time()

data = data.dropna(how='any')
print("Dataframe rows after NaN dropping: " + str(data.count()))

null_removal_finish_time = tm.time() - common_start_time
print_and_save_time("Null values removal concluded: " +
                    str(null_removal_finish_time) + " seconds")

#### Data Analysis

In [ ]:
plot_balancing_chart(data, problem_to_solve)

In [ ]:
plot_problematic_flights_per_carrier_chart(data, problem_to_solve)

In [ ]:
plot_problematic_flights_per_origin_box(data, problem_to_solve)


Based on the box plot, for the CANCELLED problem, we decided to aggregate in one column the airports with a percentage lower than 5 to improve chart readability.
For the DIVERTED problem, the value 1 is enough.

In [ ]:
plot_problematic_flights_per_origin_chart(data, problem_to_solve)

In [ ]:
plot_problematic_flights_per_month_chart(data, problem_to_solve)

In [ ]:
plot_problematic_flights_per_weekday_chart(data, problem_to_solve)

From the various charts plotted, we can see that, despite some outliers, the features don't seem to show any significant correlation with the cancellation or diversion of the flights.
The model will probably have a hard time differentiating between problematic and regular flights, due to the scarce impact of the features to determine the result of a flight.

#### Dataframe Balancing

Due to the really low amount of problematic flights found in the dataset, the training set would be really
unbalanced and probably lead to poor results, so to balance it we proceeded using the undersampling technique, limiting the number of normal flights to be taken.

In [ ]:
start_time = tm.time()
irregular_flights = data.filter(col(problem_to_solve) == 1)

regular_flights = data.filter(col(problem_to_solve) == 0).limit(irregular_flights.count())

flight_ids = irregular_flights.rdd.map(lambda x: x.id).collect() + \
    regular_flights.rdd.map(lambda x: x.id).collect()

data = data.filter(data.id.isin(flight_ids)).orderBy(rand())
print("Balanced dataframe rows: " + str(data.count()))

finish_time = tm.time() - start_time
print_and_save_time("Dataset balancing concluded: " +
                    str(finish_time) + " seconds")

In [ ]:
plot_balancing_chart(data, problem_to_solve)

#### Column Conversions

The string and timestamps columns have been converted into float numbers, to keep the values between 0 and 1 various multipliers have been used.

In [ ]:
columns_start_time = tm.time()

@udf(returnType=DoubleType())
def str_to_float(s: str):
  encoding = "utf-8"
  b = s.encode(encoding)
  return float(crc32(b) & 0xffffffff) / 2**32

date_multiplier: float = 1 / 365
@udf(returnType=DoubleType())
def date_to_day_of_year(date_string) -> float:
  date = dt.strptime(date_string, "%Y-%m-%d")
  day = date.timetuple().tm_yday - 1
  return day * date_multiplier

@udf(returnType=DoubleType())
def time_to_interval(time) -> float:
  t = int(float(time))
  h = t // 100
  m = t % 100
  t = h * 60 + m
  return float(t / 1140)

distance_multiplier = float(1) / float(max_distance)

data = data.select(
  (data.CANCELLED.cast('double')).alias("CANCELLED"),
  (data.DIVERTED.cast('double')).alias("DIVERTED"),
  str_to_float(data.OP_CARRIER).alias("OP_CARRIER"),
  str_to_float(data.ORIGIN).alias("ORIGIN"),
  str_to_float(data.DEST).alias("DEST"),
  date_to_day_of_year(data.FL_DATE).alias("FL_DATE"),
  time_to_interval(data.CRS_DEP_TIME).alias("CRS_DEP_TIME"),
  time_to_interval(data.CRS_ARR_TIME).alias("CRS_ARR_TIME"),
  time_to_interval(data.CRS_ELAPSED_TIME).alias("CRS_ELAPSED_TIME"),
  (data.DISTANCE.cast('double') * distance_multiplier).alias("DISTANCE"),
  data.id
)

data.count()

columns_finish_time = tm.time() - columns_start_time
print_and_save_time("Columns conversion concluded: " +
                    str(columns_finish_time) + " seconds")

#### Z Score Normalization

The various columns have been normalized using the z-score method, subtracting every value for the mean of its column and dividing by the column standard deviation.

In [ ]:
z_start_time = tm.time()
column_list = data.columns
column_mean_dict = dict()
column_stddv_dict = dict()

for c in column_list:
    column_mean_dict[c] = data.agg({c: 'mean'}).head()[0]
    column_stddv_dict[c] = data.agg({c: 'stddev'}).head()[0]

data = data.select(
  problem_to_solve,

  ((data.OP_CARRIER - column_mean_dict["OP_CARRIER"]) / column_stddv_dict["OP_CARRIER"]).alias('OP_CARRIER'),

  ((data.ORIGIN - column_mean_dict["ORIGIN"]) / column_stddv_dict["ORIGIN"]).alias('ORIGIN'),

  ((data.DEST - column_mean_dict["DEST"]) / column_stddv_dict["DEST"]).alias('DEST'),

  ((data.FL_DATE - column_mean_dict["FL_DATE"]) / column_stddv_dict["FL_DATE"]).alias('FL_DATE'),

  ((data.CRS_DEP_TIME - column_mean_dict["CRS_DEP_TIME"]) / column_stddv_dict["CRS_DEP_TIME"]).alias('CRS_DEP_TIME'),

  ((data.CRS_ARR_TIME - column_mean_dict["CRS_ARR_TIME"]) /  column_stddv_dict["CRS_ARR_TIME"]).alias('CRS_ARR_TIME'),

  ((data.CRS_ELAPSED_TIME - column_mean_dict["CRS_ELAPSED_TIME"]) / column_stddv_dict["CRS_ELAPSED_TIME"]).alias('CRS_ELAPSED_TIME'),

  ((data.DISTANCE - column_mean_dict["DISTANCE"]) / column_stddv_dict["DISTANCE"]).alias('DISTANCE'),

  data.id
)

data.count()

z_finish_time = tm.time() - z_start_time
print_and_save_time("Z score normalization concluded: " +
                    str(z_finish_time) + " seconds")

#### Preprocessed dataset Saving/Loading

In [ ]:
def save_dataset(data):
  data.write.format('csv').option('header', True).mode('overwrite').option(
      'sep', ',').save('./preprocessed/' + problem_to_solve)
  print('Preprocessed dataset saved')

save_dataset(data)

In [ ]:
def load_dataset():
    data = spark.read.format("csv") \
        .option("header", True) \
        .load('./preprocessed/' + problem_to_solve)

    print('Preprocessed dataset loaded')
    return data


if load_cached:
    data = load_dataset().cache()

#### Splitting into Folds

During the main phases of preprocessing the Dataframe structure have been preferred, to use the columns name notation and the sql functions to perform transformations.
The training part though found the RDD data structures to be more suitable for the tasks required, so a conversion has been accomplished.

In [ ]:
start_time = tm.time()
folds: list[RDD] = []

def format_rdd(rdd: RDD) -> RDD:
    return rdd.map(lambda x: (float(x[0]), [float(y) for y in x[1:-1]]))

temp = data

k_elements_half_number = math.floor((temp.count() / fold_number) / 2)

i = 0
while i < fold_number:
    k_positives = temp.where(
        col(problem_to_solve) == 1).limit(k_elements_half_number)

    k_negatives = temp.where(
        col(problem_to_solve) == 0).limit(k_elements_half_number)

    k_ids = k_positives.rdd.map(lambda x: x.id).collect() + \
        k_negatives.rdd.map(lambda x: x.id).collect()

    k_sample = temp.filter(temp.id.isin(k_ids))
    k_sample = k_sample.drop(k_sample.id)

    k_sample = format_rdd(k_sample.rdd)

    folds.append(k_sample)
    temp = temp.filter(~temp.id.isin(k_ids))

    print("Split " + str(i + 1) + " of " + str(fold_number) + " completed")
    print("Dataframe rows: " + str(temp.count()))
    i += 1

finish_time = tm.time() - start_time
print_and_save_time("Dataset splitting concluded: " +
                    str(finish_time) + " seconds")


#### Bonus: Pandas

To make a comparison between PySpark parallel processing and a normal sequential data processing, dataset loading and preprocessing has been performed also with the Pandas library.

In [ ]:
# Data Load
loading_start_time = tm.time()

files = os.listdir(path)
data = pd.DataFrame()

for f in files:
    if f.endswith('.csv'):
        frame = pd.read_csv(filepath_or_buffer=path +
                            '/' + f, usecols=columns_to_get)
        frame.sample(frac=1)
        frame = frame.head(dataset_limit)
        data = pd.concat([data, frame])

loading_finish_time = tm.time() - loading_start_time
print_and_save_time("Pandas dataset acquisition concluded: " +
                    str(loading_finish_time) + " seconds")

#Null values removal
null_start_time = tm.time()

data = data.dropna(how='any', axis='index')

null_finish_time = tm.time() - null_start_time
print_and_save_time("Pandas null values removal concluded: " +
                    str(null_finish_time) + " seconds")

# Problem Selection
selection_start_time = tm.time()

irregulars = data.loc[data[problem_to_solve] == 1]
regulars = data.loc[data[problem_to_solve] == 0]

data = pd.concat([regulars.sample(len(irregulars)), irregulars]).sample(frac=1)

oppositeIndex = 'DIVERTED' if problem_to_solve == 'CANCELLED' else 'CANCELLED'
data = data.drop(oppositeIndex, axis=1)

selection_finish_time = tm.time() - selection_start_time
print_and_save_time("Pandas dataset balancing concluded: " +
                    str(selection_finish_time) + " seconds")

conversion_start_time = tm.time()

# Names Conversion

def str_to_float(s: str):
    encoding = "utf-8"
    b = s.encode(encoding)
    return float(crc32(b) & 0xffffffff) / 2**32

for c in ['OP_CARRIER', 'ORIGIN', 'DEST']:
    data[c] = data[c].apply(str_to_float)

# Dates Conversion

multiplier: float = 1 / 365

def date_to_day_of_year(date_string) -> float:
    date = dt.strptime(date_string, "%Y-%m-%d")
    day = date.timetuple().tm_yday - 1
    return day * multiplier

data["FL_DATE"] = data["FL_DATE"].apply(date_to_day_of_year)

# Time Conversion
    
def time_to_interval(time) -> float:
    t = int(float(time))
    h = t // 100
    m = t % 100
    t = h * 60 + m
    return float(t / 1140)

for c in ["CRS_DEP_TIME", "CRS_ARR_TIME", "CRS_ELAPSED_TIME"]:
    data[c] = data[c].apply(time_to_interval)

# Distance Conversion
    
multiplier: float = float(1) / float(max_distance)

data["DISTANCE"] = data["DISTANCE"].apply(lambda x: x * multiplier)

conversion_finish_time = tm.time() - conversion_start_time
print_and_save_time("Pandas dataset conversions concluded: " +
                    str(conversion_finish_time) + " seconds")

normalization_start_time = tm.time()

#Z-score normalization

def z_score_normalize(x, m, s) -> float:
    return (x - m) / s

column_list = list(data)
column_list.remove(problem_to_solve)

for c in column_list:
    column_mean = data[c].mean()
    column_stddv = data[c].std()
    data[c] = data[c].apply(z_score_normalize, args=(column_mean, column_stddv))

normalization_finish_time = tm.time() - normalization_start_time
print_and_save_time("Pandas dataset normalization concluded: " +
                    str(normalization_finish_time) + " seconds")

fold_start_time = tm.time()
# Create Folds

folds = []

data = data.drop_duplicates()

irregulars = data.loc[data[problem_to_solve] == 1]
regulars = data.loc[data[problem_to_solve] == 0]

k_elements_half_number = round((len(data) / fold_number) / 2)

for i in range(1, fold_number + 1):
    k_irregulars_sample = irregulars.head(k_elements_half_number)
    k_regulars_sample = regulars.head(k_elements_half_number)
    k_sample = pd.concat([k_irregulars_sample, k_regulars_sample])

    folds.append(k_sample)
    irregulars = irregulars.drop(k_irregulars_sample.index)
    regulars = regulars.drop(k_regulars_sample.index)

fold_finish_time = tm.time() - fold_start_time
print_and_save_time("Pandas dataset folding concluded: " +
                    str(fold_finish_time) + " seconds")

## Models

### Generic Functions

In [ ]:
def sigmoid(x):
    '''
    Calculates the sigmoid of the given data
    '''
    g = 1.0 / (1.0 + np.exp(-x))
    return g

def binary_cross_entropy(y, y_label, w, l2):
    '''
    Calculates the binary cross entropy loss of the calculated y and the given y_label
    '''
    loss = -np.mean(y_label*(np.log(y)) + (1-y_label)
                    * np.log(1-y)) + regularize(w, l2)
    return loss

def regularize(W, l2):
    '''
    Calculates the regularization term for the loss
    '''
    return (l2 / 2) * np.sum(np.square(W))

### Parallel Model

In [ ]:
@dataclass
class ParallelLogisticRegression:
    iterations: int
    learning_rate: float
    batch_size: int
    l2: float
    W: Broadcast
    b: float

def parallel_initialize(self: ParallelLogisticRegression, feature_number: int):
    self.W = context.broadcast(np.random.rand(feature_number))
    self.b = np.random.rand()

def parallel_train(self: ParallelLogisticRegression, data: RDD):

    if self.batch_size != 0:
        num_chunks = data.count() // self.batch_size
        chunk_percent = 1/num_chunks
        batches = data.randomSplit([chunk_percent] * num_chunks)
    else:
        batches = [data]

    losses = []
    gradients = []

    for _ in range(self.iterations):
        _losses = []
        _gradients = []

        for batch in batches:
            batch = batch.cache()
            Y = parallel_evaluate(self, batch)
            _losses.append(parallel_binary_cross_entropy(self, batch.map(lambda x: x[0]).zip(Y)))
            (dW, db) = parallel_gradient(self, batch, Y)
            _gradients.append(dW)
            parallel_update(self, dW, db)
        losses.append(np.mean(_losses))
        gradients.append(np.mean(_gradients))

    return (losses, gradients)


def parallel_evaluate(self: ParallelLogisticRegression, X: RDD) -> RDD:
    Z: RDD = X.map(lambda x: np.dot(x[1], self.W.value))
    Z = Z.map(lambda x: sigmoid(x))
    return Z

def parallel_binary_cross_entropy(self: ParallelLogisticRegression, X_Y: RDD)-> float:
    L: RDD = X_Y.map(lambda y: y[0] * np.log(y[1]) + (1 - y[0]) * np.log(1 -  y[1]))
    return -L.reduce(lambda a, b: a + b)/L.count() + regularize(self.W.value, self.l2)

def parallel_gradient(self: ParallelLogisticRegression, X: RDD, Y: RDD)-> tuple[np.ndarray, np.ndarray]:
    m = X.count()
    dw = X.zip(Y).map(lambda x: np.dot((x[1] - x[0][0]), x[0][1])).reduce(lambda a, b: (a + b) * 1/m + self.W.value * self.l2)
    db = X.zip(Y).map(lambda x: x[1] - x[0][0]).reduce(lambda a, b: a + b) * 1/m
    return dw, db

def parallel_update(self: ParallelLogisticRegression, dW: list[float], db: float):
    self.W = context.broadcast(self.W.value - self.learning_rate * dW)
    self.b = self.b - self.learning_rate * db

### Serial Model

In [ ]:
class SerialLogisticRegression():
    def __init__(self, iterations: int, learning_rate: float, batch_size: int, l2: float):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.l2 = l2

    def initialize(self, columns_number):
        self.W = np.random.rand(columns_number)
        self.b = np.random.rand()

    def evaluate(self, X):
        Z = np.dot(X, self.W) + self.b
        Z = sigmoid(Z)
        return Z

    def gradient(self, X, Y, Y_label):
        '''
        Calculates the gradient w.r.t weights and bias
        '''

        # Number of training examples.
        m = X.shape[0]

        # Gradient of loss w.r.t weights with regularization
        dw = (1/m)*np.dot(X.T, (Y - Y_label)) + self.l2 * self.W

        # Gradient of loss w.r.t bias with regularization
        db = (1/m)*np.sum((Y - Y_label))

        return dw, db

    def update(self, dW, db):
        self.W = self.W - self.learning_rate * dW
        self.b = self.b - self.learning_rate * db

    def train(self, X, Y_labels):
        losses = []
        gradients = []

        if self.batch_size == 0:
            self.batch_size = X.shape[0]

        for _ in range(self.iterations):
            _losses = []
            _gradients = []
            for b in range(X.shape[0]//self.batch_size):
                b_X = X[b*self.batch_size:b*self.batch_size+self.batch_size, :]
                b_Y_labels = Y_labels[b*self.batch_size:b *
                                      self.batch_size+self.batch_size]
                Y = self.evaluate(b_X)
                _losses.append(binary_cross_entropy(
                    Y, b_Y_labels, self.W, self.l2))
                (dW, db) = self.gradient(b_X, Y, b_Y_labels)
                _gradients.append(dW)
                self.update(dW, db)
            losses.append(np.mean(_losses))
            gradients.append(np.mean(_gradients))

        return (losses, gradients)

## Experiments


In [ ]:
from sklearn import metrics

def describe(labels: RDD, predictions: RDD):
    zipped = labels.zip(predictions)
    tn = zipped.filter(lambda x: round(x[0]) == 0 and round(x[1]) == 0).count()
    tp = zipped.filter(lambda x: round(x[0]) == 1 and round(x[1]) == 1).count()
    fn = zipped.filter(lambda x: round(x[0]) == 1 and round(x[1]) == 0).count()
    fp = zipped.filter(lambda x: round(x[0]) == 0 and round(x[1]) == 1).count()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    acc = (tp + tn)/(tp + tn + fn + fp)
    specificity = tn/(tn+fp)
    auroc = metrics.roc_auc_score(labels.map(lambda x: round(x)).collect(), predictions.map(lambda x: round(x)).collect())
    return pd.DataFrame([[tn, tp, fn, fp, precision, recall, f1, acc, specificity, auroc]], columns=['TN', 'TP', 'FN', 'FP', 'Precision', 'Recall', 'F1', 'Accuracy', 'Specificity', "AUROC"])

In [ ]:
def make_roc(labels: RDD, results: RDD):
    labels = labels.collect()
    results = results.collect()
    labels_and_results = sorted(list(zip(labels, results)), key=lambda x: x[1])

    labels_by_weights = np.array([k for (k, _) in labels_and_results])

    length = labels_by_weights.size

    true_positives = labels_by_weights.cumsum()

    num_positive = true_positives[-1]

    false_positives = np.arange(1.0, length + 1, 1.) - true_positives

    true_positives_rate = true_positives / num_positive
    false_positives_rate = false_positives / (length - num_positive)

    fig, ax = plt.subplots()
    ax.set_xlim(-.05, 1.05), ax.set_ylim(-.05, 1.05)
    ax.set_ylabel('True Positive Rate (Sensitivity)')
    ax.set_xlabel('False Positive Rate (1 - Specificity)')
    plt.plot(false_positives_rate, true_positives_rate,
             color='#8cbfd0', linestyle='-', linewidth=3.)
    plt.plot((0., 1.), (0., 1.), linestyle='--',
             color='#d6ebf2', linewidth=2.)
    plt.show()

def plot_loss_gradient(iterations, train_losses, gradients):
    fig, ax = plt.subplots()
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Loss/Gradient')
    ax.plot(range(iterations), train_losses, label='Loss')
    ax.plot(range(iterations), gradients, label='Gradient')
    ax.grid()
    ax.legend()

    fig.clear()
    plt.close()

In [ ]:
import plotly.graph_objects as go

def make_3d(results: list):
    chart_data = [np.array([r[0].iterations, r[0].lr, r[0].l2, r[3]]) for r in results]
    chart_data = pd.DataFrame(chart_data, columns= ['iterations', 'lr', 'l2', 'loss'])

    fig = go.Figure(data=[go.Scatter3d(
        x=chart_data['lr'],
        y=chart_data['l2'],
        z=chart_data['iterations'],
        mode='markers',
        text=('lr', 'l2', 'iterations'),
        marker=dict(
            color=chart_data['loss'], 
            colorscale='Viridis',
            colorbar=dict(thickness=20)
        )
    )])

    fig.update_layout(
        autosize=False,
        width=600, 
        height=600,
        title='', 
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis_title='LR',
            yaxis_title='L2',
            zaxis_title='Iterations',
        ),
    )

    fig.show()

### Hyperparameters Tuning
Here a class to tune the hyperparameters is defined. It iterates a grid of parameters and trains a model for each configuration

In [ ]:
import itertools

grid: dict[str, list] = { 'iter': [10, 20, 50], 'lr': [0.001, 0.01, 0.1], 'l2': [1, 0.1, 0.01]}

class Tuner:
    def __init__(self, grid: dict, model_factory) -> None:
        self.params = list(itertools.product(*grid.values()))
        self.model_factory = model_factory

    def search(self, input: list[RDD]):
        results = list()
        for i in self.params:
            model = self.model_factory(*i)
            time, predictions, test_loss, train_losses = model.train(input)
            print("Completed configuration " + '|'.join(map(lambda x: str(x), i)))
            results.append((model, time, predictions, test_loss, train_losses))
        return results


#### K-Fold Cross Validation
The following code defines a base class with train and evaluation methods to apply the K-Fold Cross Validation to each model. 

In [ ]:
class KFoldTrainer:
    def __init__(self, iterations, lr, l2, batch_size):
        self.iterations = iterations
        self.lr = lr
        self.l2 = l2
        self.batch_size = batch_size

    def train(self, data: list[RDD]):
        test_set = data[0]
        rest = data[1:]
        train_losses = []
        times = []
        for i, fold in enumerate(rest):
            evaluation_set = fold
            remaining_folds= rest[:i] + rest[i + 1:]
            train_set = remaining_folds[0]
            for train_fold in remaining_folds[1:]:
                train_set = train_set.union(train_fold)
            t = tm.time()
            self.train_impl(train_fold)
            times.append(tm.time() - t)
            _, loss = self.test_impl(evaluation_set)
            train_losses.append(loss)
        predictions, test_loss = self.test_impl(test_set)
        return sum(times), predictions, test_loss, train_losses

### Parallel Implementation

In [ ]:
class ParallelModelEvaluator(KFoldTrainer):
    def __init__(self, iterations, lr, l2, batch_size):
        super().__init__(iterations, lr, l2, batch_size)
        self.model = ParallelLogisticRegression(self.iterations, self.lr, self.batch_size, self.l2, None, None)
        parallel_initialize(self.model, 7)

    def train_impl(self, train_data: RDD):
        return parallel_train(self.model, train_data)
    
    def test_impl(self, test_data: RDD):
        value: RDD = parallel_evaluate(self.model, test_data)
        return value, parallel_binary_cross_entropy(self.model, test_data.map(lambda x: x[0]).zip(value))

In [ ]:
t = Tuner(grid, lambda it, lr, l2: ParallelModelEvaluator(it, lr, l2, 0))
par_results = t.search(folds)
 
par_best = min(par_results, key=lambda x: x[3])
par_best_model = par_best[0]
par_best_loss = par_best[3]

print("Iterations {} Lr {} L2 {} Loss {}".format(par_best_model.iterations, par_best_model.lr, par_best_model.l2, par_best_loss))
print("Total time: {}".format(sum([r[1] for r in par_results])))

The following is a representation of the loss for each possible configuration. The loss is represented with a color

In [ ]:
predictions = par_best[2]
labels = folds[0].map(lambda x: x[0]).coalesce(1)
predictions = predictions.coalesce(1)
describe(labels, predictions)

In [ ]:
make_3d(par_results)

### Sequential Implementation

In [ ]:
class SequentialEvaluator(KFoldTrainer):
    def __init__(self, iterations, lr, l2, batch_size):
        super().__init__(iterations, lr, l2, batch_size)
        self.model = SerialLogisticRegression(self.iterations, self.lr, self.batch_size, self.l2)
        self.model.initialize(7)

    def train_impl(self, train_data: RDD):
        return self.model.train(np.array(train_data.map(lambda x: x[1]).collect()), np.array(train_data.map(lambda x: x[0]).collect()))

    def test_impl(self, test_data: RDD):
        res = self.model.evaluate(np.array(test_data.map(lambda x: x[1]).collect()))
        return res, binary_cross_entropy(res, np.array(test_data.map(lambda x: x[0]).collect()), self.model.W, self.l2)


In [ ]:
t = Tuner(grid, lambda it, lr, l2: SequentialEvaluator(it, lr, l2, 0))

seq_results = t.search(folds)
 
seq_best = min(seq_results, key=lambda x: x[3])
seq_best_model = seq_best[0]
seq_best_loss = seq_best[3]

print("Best Model: Iterations {} Lr {} L2 {} Loss {} in {}s".format(seq_best_model.iterations, seq_best_model.lr, seq_best_model.l2, seq_best_loss, seq_best[1]))
print("Total time: {}".format(sum([r[1] for r in seq_results])))

In [ ]:
predictions = seq_best[2]
labels = folds[0].map(lambda x: x[0]).coalesce(1)
predictions = context.parallelize(predictions).coalesce(1)
describe(labels, predictions)

In [ ]:
make_roc(labels, predictions)

In [ ]:
make_3d(seq_results)

## Mini Batch
The following experiment consists in using the mini batch feature using the best hyperparameters to find differences

In [ ]:
mini_batch_eval = SequentialEvaluator(20, 0.1, 0.01, 500)
time, mb_predictions, mb_test_loss, mb_train_losses = mini_batch_eval.train(folds)
print("Mini Batch Impl: {} in {}s".format(mb_test_loss, time))

In [ ]:
labels = folds[0].map(lambda x: x[0]).coalesce(1)
predictions = context.parallelize(mb_predictions).coalesce(1)
describe(labels, predictions)

# Pyspark ML Implementations
The following cells use some state-of-the-art models provided by the Pyspark ML library, in particular their implementation of the Logistic Regression and Decision Tree models.

They should be most performant implementations of a distributed machine learning model

In [ ]:
from pyspark.ml.feature import VectorAssembler

ml_data = data.select([col(c).cast('float').alias(c) if c != problem_to_solve else col(c).cast('float').alias('label') for c in data.columns[:-1]])

assembler = VectorAssembler(inputCols=ml_data.columns[1:-1], outputCol='features')

ml_data = assembler.transform(ml_data).select(['features', 'label'])

train_set, test_set = ml_data.randomSplit([0.9, 0.1])

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

class SparkMLTuner:
    def __init__(self, estimator, grid_builder: ParamGridBuilder, num_fold: int):
        self.evaluator = BinaryClassificationEvaluator()
        self.cv = CrossValidator(estimator=estimator, estimatorParamMaps=grid_builder.build(), evaluator=self.evaluator, numFolds=num_fold)

    def train(self, data: ps.DataFrame):
        self.model = self.cv.fit(data)
        return self.model.bestModel.params

    def test(self, data: ps.DataFrame):
        predictions = self.model.transform(data)
        return predictions, self.evaluator.evaluate(predictions)

### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

e = LogisticRegression(featuresCol='features', labelCol='label')
g = ParamGridBuilder().addGrid(e.regParam, grid['l2']).addGrid(e.maxIter, grid['iter'])
t1 = SparkMLTuner(e, g, 10)
t1.train(train_set)
predictions, loss = t1.test(test_set)

labels = test_set.select(col('label')).rdd.map(lambda x: float(x[0]))
predictions = predictions.select(col('prediction')).rdd.map(lambda x: float(x[0]))

print("Loss " + str(loss))

describe(labels, predictions)

In [ ]:
make_roc(labels, predictions)

Next the weights from the Pyspark ML model and the naive implementation are compared to see their difference

In [ ]:
pyspark_ml_weights = t1.model.bestModel.coefficients

par_impl_weights = seq_best[0].model.W 

print(par_impl_weights)
print(pyspark_ml_weights)

np.allclose(pyspark_ml_weights, par_impl_weights, rtol=0.1, atol=0.1)

### Decision Tree

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
e = DecisionTreeClassifier(featuresCol='features', labelCol='label')
g = ParamGridBuilder()
t = SparkMLTuner(e, g, 10)
t.train(train_set)
predictions, loss = t.test(test_set)

labels = test_set.select(col('label')).rdd.map(lambda x: float(x[0]))
predictions = predictions.select(col('prediction')).rdd.map(lambda x: float(x[0]))

print("Loss " + str(loss))

describe(labels, predictions)

In [ ]:
make_roc(labels, predictions)